In [ ]:
from models import SmallerVGGNet
import VipsML

base="./images"

In [ ]:
# Original images. Any VIPS-supported format of any size.
origs=[base+'/0'+str(n)+'-orig.tif' for n in range(1,4)]
# Annotated dataset. One channel, one colour per class (incremental from 0).
# Any VIPS-supported format, size same as original.
masks=[base+'/0'+str(n)+'-mask.tif' for n in range(1,4)]

# Either pixel wise or pooled depending on frame_size. Here pooling 3x3 pixels.
# meta_ratio also gives a snapshot at a lower resolution (5x) of neighboring tissue.
images = [VipsML.VipsClassificationML(orig,mask,frame_size=3, padding=30) 
                              for orig, mask in zip(origs,masks)]

# Generator takes a list of VipsML-images and wraps a supply chain for the  
# fit_generator around it.
generator = VipsML.VipsGroupML(images, batch_size=400)

# To get sensible validation data, split dataset, keeping 10% for validation.
# Re-shuffled after each epoch.
trainer, validator = generator.split_generators(0.1)

# Load and compile model
model = SmallerVGGNet(generator.input_shape, generator.classes)
model.compile(loss="categorical_crossentropy", optimizer='sgd')

# Fit model
# 163s / epoch, 5x meta, new method
# 142s / epoch, no meta, new method
# 4421s / epoch, no meta, old method
model.fit(trainer,validation_data=validator,epochs=5)